In [52]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
from colorthief import ColorThief
import io
import pickle

%matplotlib inline

In [2]:
client = MongoClient('localhost', 27017)
db = client.fashion

In [3]:
collection = db.fashion
data = pd.DataFrame(list(collection.find()))

In [4]:
df = data.copy()
df.shape

(1001, 10)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 10 columns):
_id              1001 non-null object
country          999 non-null object
credit           1000 non-null object
date             1001 non-null object
description      588 non-null object
image            1001 non-null object
medium           1000 non-null object
name             1001 non-null object
object number    1001 non-null object
rgba_arr         1001 non-null object
dtypes: object(10)
memory usage: 78.3+ KB


In [6]:
sorted(df['date'].unique())

['1900-1905',
 '1903',
 '1904',
 '1905-1915',
 '1907',
 '1908',
 '1912',
 '1913',
 '1914-1918',
 '1915-1920',
 '1923',
 '1924',
 '1925',
 '1926',
 '1926-1927',
 '1927',
 '1928',
 '1929',
 '1929-1930',
 '1930',
 '1930-1932',
 '1930s',
 '1932',
 '1933',
 '1933 - 1940',
 '1934',
 '1935',
 '1936',
 '1936-1942',
 '1937',
 '1938',
 '1939',
 '1940',
 '1940s',
 '1941',
 '1942',
 '1942-1943',
 '1944',
 '1944-1945',
 '1945',
 '1945-1949',
 '1945-1955',
 '1946',
 '1947',
 '1948-1950',
 '1950',
 '1950-1951',
 '1950s',
 '1951',
 '1951-1952',
 '1952',
 '1953',
 '1953-1954',
 '1953-1955',
 '1954',
 '1954-1955',
 '1954-56',
 '1955',
 '1955-1959',
 '1956',
 '1956-1957',
 '1957',
 '1958',
 '1958-1959',
 '1960',
 '1960-1963',
 '1960-1964',
 '1960s',
 '1961',
 '1962',
 '1963',
 '1963-1964',
 '1964',
 '1964-1965',
 '1965',
 '1965-1967',
 '1965-1968',
 '1965-1970',
 '1965-1972',
 '1966',
 '1966-1967',
 '1967',
 '1967-1968',
 '1967-1969',
 '1968',
 '1968-1969',
 '1968-1971',
 '1969',
 '1970',
 '1970-1971',
 

In [7]:
df['date'] = df['date'].str.strip()

In [8]:
mask = df['date'].str.contains('c.')
df['circa'] = mask

In [9]:
df['date'] = df['date'].str.replace('c. ', '')

In [10]:
df['date'] = df['date'].str.replace('c.', '')

In [11]:
seasons = ['Spring','Summer', 'Winter', 'Fall', 'Autumn']
mask = df['date'].str.contains('|'.join(seasons))

In [12]:
for ssn in seasons:
    mask = df['date'].str.contains(ssn)
    df[str(ssn)] = mask

In [13]:
# df['date'] = df['date'].str.strip()

In [14]:
df['date'] = df['date'].str.replace('/', '')

In [15]:
for ssn in seasons:
    df['date'] = df['date'].str.replace(ssn, '')

In [16]:
for ssn in seasons:
    df['date'] = df['date'].str.replace(ssn.lower(), '')

In [17]:
df['date'] = df['date'].str.replace('Early 20th ntury', '1900-1919')

In [18]:
df['date'] = df['date'].str.replace('late', '')

In [19]:
df['date'] = df['date'].str.replace('Resort', '')

In [20]:
df['date'] = df['date'].str.replace('r', '')

In [21]:
df['date'] = df['date'].str.replace('s', '')

In [22]:
df['date'] = df['date'].str.replace(' ', '')

In [23]:
sorted(df['date'].unique())

['1888',
 '1900',
 '1900-1905',
 '1900-1919',
 '1902',
 '1903',
 '1904',
 '1905',
 '1905-1915',
 '1907',
 '1908',
 '1909',
 '1910',
 '1910-1920',
 '1912',
 '1913',
 '1914-1918',
 '1915',
 '1915-1920',
 '1916',
 '1917',
 '1918',
 '1920',
 '1922',
 '1923',
 '1924',
 '1925',
 '1926',
 '1926-1927',
 '1927',
 '1928',
 '1929',
 '1929-1930',
 '1930',
 '1930-1932',
 '1931',
 '1932',
 '1933',
 '1933-1940',
 '1934',
 '1935',
 '1936',
 '1936-1942',
 '1937',
 '1938',
 '1939',
 '1940',
 '1941',
 '1942',
 '1942-1943',
 '1943',
 '1944',
 '1944-1945',
 '1945',
 '1945-1949',
 '1945-1955',
 '1946',
 '1947',
 '1948',
 '1948-1950',
 '1949',
 '1950',
 '1950-1951',
 '1951',
 '1951-1952',
 '1952',
 '1952-1953',
 '1953',
 '1953-1954',
 '1953-1955',
 '1954',
 '1954-1955',
 '1954-56',
 '1955',
 '1955-1959',
 '1956',
 '1956-1957',
 '1957',
 '1958',
 '1958-1959',
 '1959',
 '1960',
 '1960-1963',
 '1960-1964',
 '1960-1965',
 '1961',
 '1962',
 '1963',
 '1963-1964',
 '1964',
 '1964-1965',
 '1965',
 '1965-1967',
 '196

In [24]:
df['year_min'] = df['date'].str[0:4]

In [25]:
sorted(df['year_min'].unique())

['1888',
 '1900',
 '1902',
 '1903',
 '1904',
 '1905',
 '1907',
 '1908',
 '1909',
 '1910',
 '1912',
 '1913',
 '1914',
 '1915',
 '1916',
 '1917',
 '1918',
 '1920',
 '1922',
 '1923',
 '1924',
 '1925',
 '1926',
 '1927',
 '1928',
 '1929',
 '1930',
 '1931',
 '1932',
 '1933',
 '1934',
 '1935',
 '1936',
 '1937',
 '1938',
 '1939',
 '1940',
 '1941',
 '1942',
 '1943',
 '1944',
 '1945',
 '1946',
 '1947',
 '1948',
 '1949',
 '1950',
 '1951',
 '1952',
 '1953',
 '1954',
 '1955',
 '1956',
 '1957',
 '1958',
 '1959',
 '1960',
 '1961',
 '1962',
 '1963',
 '1964',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1971',
 '1972',
 '1973',
 '1974',
 '1975',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 

In [26]:
df['decade'] = df['year_min'].str[0:3] + '0'

In [27]:
sorted(df['decade'].unique())

['1880',
 '1900',
 '1910',
 '1920',
 '1930',
 '1940',
 '1950',
 '1960',
 '1970',
 '1980',
 '1990',
 '2000',
 '2010']

In [28]:
# lengths = list()
# for i in range(1001):
#     ratio = len(df.loc[i]['rgba_arr'])
#     if ratio % 3 != 0 or ratio % 4 != 0:
#         print(i, ratio)

In [29]:
# df.drop(df.index[[156]], inplace=True)

In [30]:
df.head()

,_id,country,credit,date,description,image,medium,name,object number,rgba_arr,circa,Spring,Summer,Winter,Fall,Autumn,year_min,decade
0,59e69af80386b0109bb37f73,USA,Gift of Mrs. Howard Serrell,1912,Meta Buehner wore this dress for her wedding t...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,"Champagne silk jacquard with metallic yarns, i...",Wedding dress,95.145.10,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,True,False,False,False,False,False,1912,1910
1,59e69b030386b0109bb37f74,USA (possibly),Gift of Elizabeth Roberts,1918,Tea gowns were worn for entertaining at home d...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,"Chartreuse silk chiffon, ecru lace, mink, pink...",Tea gown,90.180.4,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,True,False,False,False,False,False,1918,1910
2,59e69b0d0386b0109bb37f75,France,Museum purchase,1900,"The tea gown became ""an indispensable element ...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Off-white silk crimped chiffon with red and gr...,Tea gown,2003.12.1,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,True,False,False,False,False,False,1900,1900
3,59e69b180386b0109bb37f76,USA,Museum purchase,1916,NaN,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Linen wool blend,Suit,P84.25.4,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,True,False,False,False,False,False,1916,1910
4,59e69b230386b0109bb37f77,USA,Museum purchase,1916,This suit follows the fashionable silhouette o...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Cotton denim,Suit,P85.35.2,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,True,False,False,False,False,False,1916,1910


In [31]:
decade_list = [decade for decade in sorted(df['decade'].unique())]

In [32]:
len(decade_list)

13

In [49]:
decade_palettes = {}

for decade in decade_list:
    decade_palettes[decade] = list(df['year_min'])
    

In [50]:
list(decade_palettes)

['1880',
 '1900',
 '1910',
 '1920',
 '1930',
 '1940',
 '1950',
 '1960',
 '1970',
 '1980',
 '1990',
 '2000',
 '2010']

In [51]:
print(decade_palettes)

{'1880': ['1912', '1918', '1900', '1916', '1916', '1913', '1910', '1903', '1907', '1915', '1907', '1900', '1910', '1914', '1908', '1904', '1909', '1913', '1888', '1904', '1910', '1900', '1910', '1904', '1912', '1900', '1900', '1905', '1908', '1905', '1918', '1915', '1915', '1917', '1905', '1915', '1907', '1913', '1903', '1902', '1915', '1903', '1937', '1937', '1925', '1930', '1935', '1936', '1939', '1936', '1927', '1930', '1935', '1930', '1938', '1939', '1925', '1939', '1935', '1932', '1930', '1925', '1933', '1935', '1932', '1920', '1934', '1933', '1935', '1935', '1937', '1927', '1920', '1930', '1924', '1928', '1928', '1923', '1928', '1930', '1935', '1937', '1937', '1937', '1936', '1934', '1935', '1929', '1929', '1936', '1931', '1920', '1939', '1938', '1937', '1935', '1933', '1933', '1936', '1936', '1932', '1935', '1930', '1924', '1938', '1932', '1938', '1930', '1929', '1936', '1926', '1934', '1938', '1927', '1930', '1928', '1938', '1936', '1935', '1926', '1939', '1938', '1933', '1926'

In [53]:
with open('../../pickle/fashion.pkl', 'rb') as f:
    df = pickle.load(f)

In [54]:
df

,_id,country,credit,date,description,image,medium,name,object number,rgba_arr,circa,Spring,Summer,Winter,Fall,Autumn,year_min,decade,palette
0,59e69af80386b0109bb37f73,USA,Gift of Mrs. Howard Serrell,1912,Meta Buehner wore this dress for her wedding t...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,"Champagne silk jacquard with metallic yarns, i...",Wedding dress,95.145.10,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,True,False,False,False,False,False,1912,1910,"[(108, 106, 105), (206, 200, 195), (167, 162, ..."
1,59e69b030386b0109bb37f74,USA (possibly),Gift of Elizabeth Roberts,1918,Tea gowns were worn for entertaining at home d...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,"Chartreuse silk chiffon, ecru lace, mink, pink...",Tea gown,90.180.4,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,True,False,False,False,False,False,1918,1910,"[(140, 133, 121), (204, 201, 198), (210, 187, ..."
2,59e69b0d0386b0109bb37f75,France,Museum purchase,1900,"The tea gown became ""an indispensable element ...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Off-white silk crimped chiffon with red and gr...,Tea gown,2003.12.1,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,True,False,False,False,False,False,1900,1900,"[(88, 87, 85), (199, 194, 187), (153, 147, 140..."
3,59e69b180386b0109bb37f76,USA,Museum purchase,1916,NaN,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Linen wool blend,Suit,P84.25.4,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,True,False,False,False,False,False,1916,1910,"[(214, 209, 203), (141, 108, 71), (188, 158, 1..."
4,59e69b230386b0109bb37f77,USA,Museum purchase,1916,This suit follows the fashionable silhouette o...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Cotton denim,Suit,P85.35.2,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,True,False,False,False,False,False,1916,1910,"[(214, 210, 207), (80, 66, 50), (153, 139, 121..."
5,59e69b2e0386b0109bb37f78,USA,Gift of Lady Arlene Kieta,1913,"After 1880, women began regularly wearing suit...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,"Brown cotton sateen, cotton floss, soutache co...",Suit,2001.58.1,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,True,False,False,False,False,False,1913,1910,"[(111, 91, 115), (96, 51, 25), (244, 230, 212)..."
6,59e69b380386b0109bb37f79,England,Gift of Martin Kamer,1910,Peter Yapp Ltd. was a prominent court shoemake...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Polychrome silk brocade with silver metal and ...,Shoes,83.208.2,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,True,False,False,False,False,False,1910,1910,"[(224, 196, 168), (159, 113, 60), (104, 58, 24..."
7,59e69b430386b0109bb37f7a,USA,Gift of Florence Anderson and Mary A. Seymour,1903,NaN,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Blue silk satin and cream lace,Petticoat,80.1.6,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,True,False,False,False,False,False,1903,1900,"[(81, 77, 72), (206, 205, 203), (143, 141, 138..."
8,59e69b4e0386b0109bb37f7b,USA,Gift of Catherine B. Canda,1907,"This delicate, revealing nightgown was made by...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,"Cotton, bobbin lace, silk ribbon",Nightgown,82.91.1,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,False,False,False,False,False,False,1907,1900,"[(128, 123, 115), (223, 216, 207), (210, 200, ..."
9,59e69b590386b0109bb37f7c,possibly USA,Gift of Armand R. Brun,1915,“No dainty woman wishes to appear untidy or un...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,"Embroidered silk net, silk ribbon, silk flowers",Night cap,96.109.1,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,True,False,False,False,False,False,1915,1910,"[(221, 217, 213), (165, 130, 74), (75, 78, 46)..."


In [55]:
df['palette'][0]

[(108, 106, 105),
 (206, 200, 195),
 (167, 162, 158),
 (30, 28, 27),
 (75, 63, 49),
 (63, 49, 38),
 (180, 180, 188),
 (180, 188, 188)]

In [57]:
df['slice_palette'] = df['palette'][1:]

In [59]:
df.to_pickle('../..pickle/sliced_fashion.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '../..pickle/sliced_fashion.pkl'